DOM vs SAX
---
操作XML有两种方法：DOM和SAX。DOM会把整个XML读入内存，解析为树，因此占用内存大，解析慢，优点是可以任意遍历树的节点。SAX是流模式，边读边解析，占用内存小，解析快，缺点是我们需要自己处理**事件**。

正常情况下，优先考虑**SAX**，因为DOM实在太占内存。

在Python中使用SAX解析XML非常简洁，通常我们关心的事件是**start_element，end_element，char_data**，准备好这3个函数，然后就可以解析xml了。

例如，当SAX解析器读到一个节点：&lt;a href="/">python&lt;/a>  
会产生3个事件：
1. start_element事件，在读取&lt;a href="/">时；
2. char_data事件，在读取python时；
3. end_element事件，在读取&lt;/a>时。


需要注意的是读取一大段字符串时，CharacterDataHandler可能被多次调用，所以需要自己保存起来，在EndElementHandler里面再合并。?

In [23]:
from xml.parsers.expat import ParserCreate

class DefaultSaxHandler(object):
    def start_element(self, name, attrs):
        print('sax:start_element: %s, attrs: %s' % (name, str(attrs)))

    def end_element(self, name):
        print('sax:end_element: %s' % name)

    def char_data(self, text):
        print('sax:char_data: %s' % text)

xml = r'''<?xml version="1.0"?>
<ol>
    <li><a href="/python">Python</a></li>
    <li><a href="/ruby">Ruby</a></li>
</ol>
'''

handler = DefaultSaxHandler()
parser = ParserCreate()
parser.StartElementHandler = handler.start_element
parser.EndElementHandler = handler.end_element
parser.CharacterDataHandler = handler.char_data
parser.Parse(xml)

sax:start_element: ol, attrs: {}
sax:char_data: 

sax:char_data:     
sax:start_element: li, attrs: {}
sax:start_element: a, attrs: {'href': '/python'}
sax:char_data: Python
sax:end_element: a
sax:end_element: li
sax:char_data: 

sax:char_data:     
sax:start_element: li, attrs: {}
sax:start_element: a, attrs: {'href': '/ruby'}
sax:char_data: Ruby
sax:end_element: a
sax:end_element: li
sax:char_data: 

sax:end_element: ol


1

除了解析XML外，如何生成XML呢？99%的情况下需要生成的XML结构都是非常简单的，因此，最简单也是最有效的生成XML的方法是拼接字符串。如果要生成复杂的XML建议不要用XML，改成JSON。

In [18]:

L = []
L.append(r'<?xml version="1.0"?>')
L.append(r'<root>')
L.append('some & data')
L.append(r'</root>')
''.join(L)

'<?xml version="1.0"?><root>some & data</root>'

小结
---
解析XML时，注意找出自己感兴趣的节点，响应事件时，把节点数据保存起来。解析完毕后，就可以处理数据。

练习
---
请利用SAX编写程序解析Yahoo的XML格式的天气预报，获取天气预报：

https://query.yahooapis.com/v1/public/yql?q=select%20*%20from%20weather.forecast%20where%20woeid%20%3D%202151330&format=xml

参数woeid是城市代码，要查询某个城市代码，可以在weather.yahoo.com搜索城市，浏览器地址栏的URL就包含城市代码。

In [37]:
from xml.parsers.expat import ParserCreate
from urllib import request

class DefaultSaxHandler(object):
    def __init__(self):
        self.res = {'city':'','forecast':[]}
        
    def start_element(self, name, attrs):
        if attrs.__contains__('city'):
            self.res['city'] = attrs['city']
        
        if 'date' in attrs and 'high' in attrs and 'low' in attrs:
            self.res['forecast'].append({'date':attrs['date'],'high':attrs['high'],'low':attrs['low']})
            
def parseXml(xml_str):
    handler = DefaultSaxHandler()
    parser = ParserCreate()
    parser.StartElementHandler = handler.start_element
    parser.Parse(xml_str)
    return handler.res

# 测试:
URL = 'https://query.yahooapis.com/v1/public/yql?q=select%20*%20from%20weather.forecast%20where%20woeid%20%3D%202151330&format=xml'

with request.urlopen(URL, timeout=4) as f:
    data = f.read()

result = parseXml(data.decode('utf-8'))
assert result['city'] == 'Beijing'
print(result)

{'city': 'Beijing', 'forecast': [{'date': '22 Mar 2018', 'high': '67', 'low': '31'}, {'date': '23 Mar 2018', 'high': '64', 'low': '39'}, {'date': '24 Mar 2018', 'high': '68', 'low': '37'}, {'date': '25 Mar 2018', 'high': '74', 'low': '40'}, {'date': '26 Mar 2018', 'high': '74', 'low': '43'}, {'date': '27 Mar 2018', 'high': '77', 'low': '46'}, {'date': '28 Mar 2018', 'high': '77', 'low': '50'}, {'date': '29 Mar 2018', 'high': '63', 'low': '49'}, {'date': '30 Mar 2018', 'high': '60', 'low': '45'}, {'date': '31 Mar 2018', 'high': '60', 'low': '43'}]}
